In [49]:
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime
from pathlib import Path

In [50]:
import xarray as xr

In [51]:
xr.DataArray(
    {"value":float(1.0)},
)

<xarray.DataArray ()> Size: 8B
array({'value': 1.0}, dtype=object)

In [2]:
hourly = xr.date_range("2025-01-01", "2025-01-02", freq="h")
minute_5 = xr.date_range("2025-01-01", "2025-01-02", freq="5min")


solar = pd.DataFrame({'time_s': hourly, 'solar': np.random.rand(hourly.size)})
air_temperature = pd.DataFrame({'time_a': minute_5, 'air_temperature': np.random.rand(minute_5.size)})
water_temperature = pd.DataFrame({'time_w': minute_5, 'water_temperature': np.random.rand(minute_5.size)})


In [3]:
ds_air = xr.Dataset({
    "air_temperature" : xr.DataArray(
            air_temperature["air_temperature"].values,
            coords={"time": air_temperature["time_a"]},
            dims=("time"),
    )}
)

In [4]:
ds_solar = xr.Dataset({
    "solar_radiation" : xr.DataArray(
            solar["solar"].values,
            coords={"time": solar["time_s"]},
            dims=("time"),
    )
})

In [7]:
ds_solar

<xarray.Dataset> Size: 400B
Dimensions:          (time: 25)
Coordinates:
  * time             (time) datetime64[ns] 200B 2025-01-01 ... 2025-01-02
Data variables:
    solar_radiation  (time) float64 200B 0.0143 0.4348 0.5392 ... 0.3822 0.447

In [5]:
ds_new = ds_air.merge(ds_solar)

In [6]:
ds_new


<xarray.Dataset> Size: 7kB
Dimensions:          (time: 289)
Coordinates:
  * time             (time) datetime64[ns] 2kB 2025-01-01 ... 2025-01-02
Data variables:
    air_temperature  (time) float64 2kB 0.6001 0.4248 0.2907 ... 0.3127 0.4313
    solar_radiation  (time) float64 2kB 0.0143 nan nan nan ... nan nan nan 0.447

In [24]:
import dask as dd
import dask.dataframe
from pathlib import Path
import datetime

In [21]:
DATA_ROOT = Path(
    r"C:\Users\ptomasula\Repositories\ClearWater-modules\data_temp\sumwere_creek_coarse_p48"
)

In [ ]:
hourly = xr.date_range("1900-01-01", "2025-01-01", freq="5min")

solar = pd.DataFrame({'time': hourly, 'solar': np.random.rand(hourly.size)})
solar.to_csv("solar.csv", index=False)

In [28]:
%%time
solar = dd.dataframe.read_csv("solar.csv")
solar.loc[solar["time"] > "01-01-2024 00:00:00"].compute()

CPU times: total: 9.02 s
Wall time: 6.23 s


,time,solar
0,1900-01-01 00:00:00,0.271629
1,1900-01-01 00:05:00,0.656735
2,1900-01-01 00:10:00,0.658174
3,1900-01-01 00:15:00,0.606963
4,1900-01-01 00:20:00,0.353705
...,...,...
1643604,2024-12-31 23:40:00,0.088204
1643605,2024-12-31 23:45:00,0.724706
1643606,2024-12-31 23:50:00,0.718755
1643607,2024-12-31 23:55:00,0.698038


In [12]:
import pandas as pd


In [37]:
f = open("solar.csv", "r")

In [38]:
%%time

def find_index(read_start, read_end):
    dt = datetime.timedelta(minutes=5)
    start= datetime.datetime(1900, 1, 1)
    read_offset = read_start - start
    read_index = read_offset // dt
    end_offset = read_end - start
    end_index = end_offset // dt
    return read_index+1, end_index+1
    
read_start = datetime.datetime(2024, 9, 1)
read_end = datetime.datetime(2025, 1, 1)
read_index, end_index = find_index(read_start, read_end)
solar_2 = pd.read_csv(f, skiprows=read_index, names=["time","solar"], nrows=end_index - read_index)
solar_2 = solar_2.loc[solar_2["time"] > "01-01-2024 00:00:00"]


CPU times: total: 3.77 s
Wall time: 3.82 s


In [34]:
%%time
solar_2 = pd.read_csv("solar.csv")
solar_2 = solar_2.loc[solar_2["time"] > "01-01-2024 00:00:00"]

CPU times: total: 6.86 s
Wall time: 6.89 s


In [ ]:
solar_2

In [ ]:
inputs = xr.Dataset(
    {
        "solar_radiation": xr.DataArray(
            solar["solar"].values,
            coords={"time": solar["time"]},
            dims=("time"),
        ),
        "air_temperature": xr.DataArray(
            air_temperature["air_temperature"].values,
            coords={"time": air_temperature["time"]},
            dims=("time"),
        ),
    },
    coords={"time": minute_5}
)

In [ ]:
target = datetime(2025,1,1,10,15)

In [ ]:
inputs["solar_radiation"].interpolate_na(dim="time", method="linear")


In [ ]:
inputs

In [ ]:
registry = {
    "var1": None,
    "solar": inputs.solar_radiation,
    "air_temperature": inputs.air_temperature,
    "water_temperature": water_temperature,
}

In [ ]:
import sys

sys.getsizeof(registry)


In [ ]:
ds = registry["solar"]
ds

In [ ]:
ts = hourly[0]
inputs.sel(time=ts)


In [39]:
#trying to understand zarr

In [42]:
hourly = xr.date_range("2025-01-01", "2025-01-02", freq="h")
minute_5 = xr.date_range("2025-01-01", "2025-01-02", freq="5min")


solar = pd.DataFrame({'time': hourly, 'solar': np.random.rand(hourly.size)})
air_temperature = pd.DataFrame({'time': minute_5, 'air_temperature': np.random.rand(minute_5.size)})
water_temperature = pd.DataFrame({'time': minute_5, 'water_temperature': np.random.rand(minute_5.size)})


In [78]:
xr_solar = xr.DataArray(
            solar["solar"].values,
            coords={"time": solar["time"]},
            dims=("time"),
        )

xr_air = xr.DataArray(
            air_temperature["air_temperature"].values,
            coords={"time": air_temperature["time"]},
            dims=("time"),
        )

In [80]:
xr_solar.to_zarr("met.zarr", append_dim="time")


In [81]:
xr_air.to_zarr("met.zarr", append_dim="time")

In [86]:
xr_met = xr.open_zarr("met.zarr")

ValueError: conflicting sizes for dimension 'time': length 289 on 'air_temperature' and length 314 on {'time': '__xarray_dataarray_variable__'}

In [54]:
xr_met

<xarray.Dataset> Size: 5kB
Dimensions:                        (time: 314)
Coordinates:
  * time                           (time) datetime64[ns] 3kB 2025-01-01 ... 2...
Data variables:
    __xarray_dataarray_variable__  (time) float64 3kB dask.array<chunksize=(25,), meta=np.ndarray>

In [88]:
inputs = xr.Dataset(
    {
        "solar_radiation": xr.DataArray(
            solar["solar"].values,
            coords={"time": solar["time"]},
            dims=("time"),
        ),
        "air_temperature": xr.DataArray(
            air_temperature["air_temperature"].values,
            coords={"time": air_temperature["time"]},
            dims=("time"),
        ),
    },
    coords={"time": minute_5}
)

In [90]:
inputs.to_zarr("met.zarr")

In [91]:
input_read = xr.open_zarr("./met.zarr")
input_read.air_temperature.time



<xarray.DataArray 'time' (time: 289)> Size: 2kB
array(['2025-01-01T00:00:00.000000000', '2025-01-01T00:05:00.000000000',
       '2025-01-01T00:10:00.000000000', ..., '2025-01-01T23:50:00.000000000',
       '2025-01-01T23:55:00.000000000', '2025-01-02T00:00:00.000000000'],
      shape=(289,), dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2kB 2025-01-01 ... 2025-01-02

In [92]:
input_2 = xr.Dataset(
    coords={"time": minute_5}
)

In [94]:
input_2.to_zarr("met2.zarr")

In [97]:
input_2 = input_2.merge({"air_temperature":xr_air})

In [100]:
input_2.to_zarr("met2.zarr")

ContainsGroupError: path '' contains a group

In [52]:
import dask.array as da

### Variable based approach

In [20]:
start_time = "2022-05-14 00:00:00"
end_time = "2022-05-15 00:00:00"
time_step = "30s"
time = pd.date_range(start=start_time, end=end_time, freq=time_step)

meterological_template = xr.Dataset(
    #{v : (("time"), da.empty(time.shape[0], dtype="float")) for v in meterological_variables},
    data_vars={},
    coords={"time": time},
)

In [294]:
file = "met2.zarr"
meterological_template.to_zarr(file, compute=False, mode='w')

Delayed('_finalize_store-d523ebde-edee-4c6e-9892-eefe861f25ba')

In [295]:
air = pd.DataFrame({'time': time, 'air_temperature': np.random.rand(time.size)})
#xr_air = xr.Dataset({'air_temperature': xr.DataArray(air['air_temperature'], dims=['time'], coords={'time': time})})
xr_air =xr.DataArray(air['air_temperature'], dims=['time'], coords={'time': time})

In [300]:
#try writing a piece
xr_air.to_zarr(f'{file}', mode="a")

In [301]:
met2 = xr.open_zarr(file)

In [305]:
met2

<xarray.Dataset> Size: 46kB
Dimensions:          (time: 2881)
Coordinates:
  * time             (time) datetime64[ns] 23kB 2022-05-14 ... 2022-05-15
Data variables:
    air_temperature  (time) float64 23kB dask.array<chunksize=(2881,), meta=np.ndarray>

### Template Approach

In [136]:
start_time = "2022-05-14 00:00:00"
end_time = "2022-05-15 00:00:00"
time_step = "30s"
time = pd.date_range(start=start_time, end=end_time, freq=time_step)

meterological_variables = [
    "air_temperature",
    "solar_radiation",
    "cloudiness",
    "wind_speed",
    "atmospheric_pressure",
    "atmospheric_vapor_pressure",
    "sediment_temperature",
    "sediment_thickness",
]

meterological_template = xr.Dataset(
    {v : (
        ("time", "scalar"), da.empty((time.shape[0],1), dtype="float"),
        ) 
        for v in meterological_variables
    },
    coords={"time": time, "scalar": [0]},
)

In [137]:
da.empty((time.shape[0],1), dtype="float").compute()

array([[5.e-324],
       [5.e-324],
       [5.e-324],
       ...,
       [5.e-324],
       [5.e-324],
       [5.e-324]], shape=(2881, 1))

In [138]:
meterological_template['air_temperature'].values

array([[5.e-324],
       [5.e-324],
       [5.e-324],
       ...,
       [5.e-324],
       [5.e-324],
       [5.e-324]], shape=(2881, 1))

In [139]:
file = "met.zarr"
meterological_template.to_zarr(file, compute=False, mode='w-')

Delayed('_finalize_store-12ee8ad1-3b57-4340-ae07-bfc45d77684f')

In [ ]:
met = xr.open_zarr(file)
met.air_temperature[4]

<xarray.DataArray 'air_temperature' (scalar: 1)> Size: 8B
dask.array<getitem, shape=(1,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * scalar   (scalar) <U6 24B 'scalar'
    time     datetime64[ns] 8B 2022-05-14T00:01:30

In [140]:
air = pd.DataFrame({
    'time': time,
    'air_temperature': np.random.rand(time.size),
    #'scalar': np.empty(time.size)
})
xr_air =xr.DataArray(air['air_temperature'], dims=['time'], coords={'time': time})
xr_air = xr_air.expand_dims({'scalar': 1})


In [141]:
xr_air

<xarray.DataArray 'air_temperature' (scalar: 1, time: 2881)> Size: 23kB
array([[0.47025278, 0.18030672, 0.12279759, ..., 0.679082  , 0.80936332,
        0.65892029]], shape=(1, 2881))
Coordinates:
  * time     (time) datetime64[ns] 23kB 2022-05-14 ... 2022-05-15
Dimensions without coordinates: scalar

In [131]:
xr_air.sel(time=time[0]).values

array([0.23480024])

In [104]:
met["air_temperature"] = xr_air


In [ ]:
met = xr.open_zarr(file)
met

In [142]:
#try writing a piece
xr_air.to_zarr(file, mode="a")

In [143]:
met = xr.open_zarr(file)

In [144]:
met

<xarray.Dataset> Size: 207kB
Dimensions:                     (time: 2881, scalar: 1)
Coordinates:
  * scalar                      (scalar) int64 8B 0
  * time                        (time) datetime64[ns] 23kB 2022-05-14 ... 202...
Data variables:
    air_temperature             (time, scalar) float64 23kB dask.array<chunksize=(2881, 1), meta=np.ndarray>
    atmospheric_pressure        (time, scalar) float64 23kB dask.array<chunksize=(2881, 1), meta=np.ndarray>
    atmospheric_vapor_pressure  (time, scalar) float64 23kB dask.array<chunksize=(2881, 1), meta=np.ndarray>
    cloudiness                  (time, scalar) float64 23kB dask.array<chunksize=(2881, 1), meta=np.ndarray>
    sediment_temperature        (time, scalar) float64 23kB dask.array<chunksize=(2881, 1), meta=np.ndarray>
    sediment_thickness          (time, scalar) float64 23kB dask.array<chunksize=(2881, 1), meta=np.ndarray>
    solar_radiation             (time, scalar) float64 23kB dask.array<chunksize=(2881, 1), meta=np.ndarray>
    wind_speed                  (time, scalar) float64 23kB dask.array<chunksize=(2881, 1), meta=np.ndarray>

In [146]:
met.air_temperature.time
met.air_temperature.values



array([[0.47025278],
       [0.18030672],
       [0.12279759],
       ...,
       [0.679082  ],
       [0.80936332],
       [0.65892029]], shape=(2881, 1))

In [32]:
met.air_temperature.values

array([nan, nan, nan, ..., nan, nan, nan], shape=(2881,))

In [147]:
#### read confirmation
file = Path("C:/Users/ptomasula/Repositories/ClearWater-modules/data_temp/dev/model_data.zarr")
met = xr.open_zarr(f'{file}/air_temperature')
met

ContainsArrayError: path '' contains an array